In [ ]:
!pip install 'aif360[LFR]' fairlearn gpflow

In [2]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

[Errno 2] No such file or directory: '/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult'
/Volumes/GoogleDrive/My Drive/Colab Notebooks


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

In [1]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras
import time

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate

import copy


from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.metrics import ClassificationMetric

import gpflow

from gpflow.utilities import print_summary, set_trainable
from gpflow.ci_utils import ci_niter

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)
tf.random.set_seed(123)

In [2]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])

train, test = dataset_orig.split([0.7], shuffle=True, seed=3)
#print("training data size", train.features.shape)
#print("dataset feature names", train.feature_names)

data_preprocessing = Pipeline([("standardiser", StandardScaler()),
                               #("minmaxscaler", MinMaxScaler()),
                                ]
                                )


# x_train = train.features
# x_test = test.features
x_train = data_preprocessing.fit_transform(train.features)
x_test = data_preprocessing.fit_transform(test.features)
y_train = train.labels.ravel()
y_test = test.labels.ravel()

#training_data = (x_train, y_train)

C = 2

N = len(x_train)
M = 400
print("Number of training datapoints:", N)
print("Number of features", len(x_train[0]))


Number of training datapoints: 34189
Number of features 18


## Model Training

### Setting up the model

In [3]:
class SVGP_classifier:
    def __init__(self, num_of_classes=2, num_of_inducing_points=50,minibatch_size=50):
        self.mb_size = minibatch_size
        self.M = num_of_inducing_points
        self.C = num_of_classes
        self.elbo_training_log = []
        self.mb_size = 50

        kernel_multi = gpflow.kernels.Matern52() + gpflow.kernels.White(variance=0.01)
        # Robustmax Multiclass Likelihood
        invlink = gpflow.likelihoods.RobustMax(self.C)  # Robustmax inverse link function
        likelihood_multi = gpflow.likelihoods.MultiClass(self.C, invlink=invlink)  # Multiclass likelihood
        Z = x_train[::self.M].copy()  # inducing inputs

        self.m = gpflow.models.SVGP(
            kernel=kernel_multi,
            likelihood=likelihood_multi,
            inducing_variable=Z,
            num_latent_gps=self.C,
            whiten=True,
            q_diag=True,
        )

        set_trainable(self.m.kernel.kernels[0].variance, False)
        set_trainable(self.m.inducing_variable, False)

    def run_adam(self, model, iterations, training_data, minibatch_size):
        """
        Utility function running the Adam optimizer

        :param model: GPflow model
        :param interations: number of iterations
        """
        # Create an Adam Optimizer action
        train_dataset = tf.data.Dataset.from_tensor_slices(training_data).repeat().shuffle(N)
        logf = []
        train_iter = iter(train_dataset.batch(minibatch_size))
        training_loss = model.training_loss_closure(train_iter, compile=True)
        optimizer = tf.optimizers.Adam()

        @tf.function
        def optimization_step():
            optimizer.minimize(training_loss, model.trainable_variables)

        for step in range(iterations):
            optimization_step()
            if step % 10 == 0:
                elbo = -training_loss().numpy()
                logf.append(elbo/minibatch_size)
        return logf
 

    def fit(self, X_train, y_train, minibatch_size=50, total_iterations=10000):
        self.mb_size = minibatch_size
        training_data = (X_train, y_train)
        maxiter = ci_niter(total_iterations)
        self.elbo_training_log = self.run_adam(self.m, maxiter, training_data, self.mb_size)

    def predict_targets(self, X_test):
        mu, _ = self.m.predict_y(X_test)
        predictions = []
        for probs in mu:
            prediction = np.argmax(probs)
            predictions.append(prediction)
        return predictions

    def get_params(self, deep=False):
        return {'minibatch_size':self.mb_size}



In [ ]:
svgpclass = SVGP_classifier(num_of_inducing_points=400)
svgpclass2 = SVGP_classifier(num_of_inducing_points=400)

logclass = LogisticRegression()

# svgpclass.fit(x_train, y_train, 50)
# predictions = svgpclass.predict_targets(x_test)
# accuracy = accuracy_score(y_test, predictions)

scores = cross_validate(svgpclass, x_train, y_train, scoring='accuracy', cv=5, fit_params={"minibatch_size": 64, "total_iterations":100})
print(scores["test_score"])


In [42]:
print(accuracy)

0.7726745376373438


In [ ]:
mb_sizes = [10]#, 50, 100, 300, 500, 1000, 5000, 10000]

#mb_sizes = np.linspace(10, 500, 20, dtype=int)

elbo_training_logs = []
models = []


with tf.device('/device:GPU:0'):
    for mb_size in mb_sizes:
        model = copy.deepcopy(m)
        elbo_training_log = run_adam(model, maxiter, training_data, mb_size)
        elbo_training_logs.append(np.array(elbo_training_log)/mb_size)
        models.append(model)


# for log in elbo_training_logs:
#     plt.plot(range(0, total_iterations, 10), log)
#     plt.show()

name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.kernels[0].variance,Parameter,Softplus,,False,(),float64,1.0
SVGP.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,0.009999999999999998
SVGP.likelihood.invlink.epsilon,Parameter,Sigmoid,Beta,False,(),float64,0.0010000000000000005
SVGP.inducing_variable.Z,Parameter,Identity,,False,"(86, 18)",float64,"[[0.41174559, 0.70430101, -0.23030082..."
SVGP.q_mu,Parameter,Identity,,True,"(86, 2)",float64,"[[0., 0...."
SVGP.q_sqrt,Parameter,Softplus,,True,"(86, 2)",float64,"[[1., 1...."


name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.kernels[0].variance,Parameter,Softplus,,False,(),float64,1.0
SVGP.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,6.227403248423943
SVGP.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,9.773503522622393e-05
SVGP.likelihood.invlink.epsilon,Parameter,Sigmoid,Beta,False,(),float64,0.0010000000000000005
SVGP.inducing_variable.Z,Parameter,Identity,,False,"(86, 18)",float64,"[[0.41174559, 0.70430101, -0.23030082..."
SVGP.q_mu,Parameter,Identity,,True,"(86, 2)",float64,"[[1.49871129e+00, -1.49871129e+00..."
SVGP.q_sqrt,Parameter,Softplus,,True,"(86, 2)",float64,"[[0.19736106, 0.19736106..."


In [ ]:





def calculate_test_accuracy(model, x_test, y_test):
    predictions = predict_targets(model, x_test)
    total_correct = 0
    for i in range(len(x_test)):
        if predictions[i]==y_test[i]:
            total_correct += 1

    return total_correct/len(predictions)


def logistic_regression_accuracy(training_data, x_test, y_test):
    model = LogisticRegression()
    model.fit(*training_data)
    predictions = model.predict(x_test)
    return accuracy_score(y_test, predictions)

def mlp_accuracy(training_data, x_test, y_test):
    model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    model.fit(*training_data)
    predictions = model.predict(x_test)
    return accuracy_score(y_test, predictions)


fairness_scores = []
test_accuracies = []

for i in range(len(models)):
    test_accuracy = calculate_test_accuracy(models[i], x_test, y_test)
    print(f"SVGP accuracy (minibatch = {mb_sizes[i]}):", test_accuracy)
    test_accuracies.append(test_accuracy)
    predictions = predict_targets(models[i], x_test)
    test_pred = test.copy()
    test_pred.labels = np.array(predictions)
    fairness_metric = ClassificationMetric(
        test, 
        test_pred,
        unprivileged_groups=unprivileged_groups, 
        privileged_groups=privileged_groups
        )
    fairness_scores.append(fairness_metric.equal_opportunity_difference())
print("Logistic Regression accuracy:", logistic_regression_accuracy(training_data, x_test, y_test))
print("MLP accuracy:", mlp_accuracy(training_data, x_test, y_test))



SVGP accuracy (minibatch = 10): 0.7530881048249505
SVGP accuracy (minibatch = 64): 0.7856411656316112
SVGP accuracy (minibatch = 118): 0.7938306148911486
SVGP accuracy (minibatch = 173): 0.7945813144066062
SVGP accuracy (minibatch = 227): 0.7945813144066062
SVGP accuracy (minibatch = 282): 0.7979253395209173
SVGP accuracy (minibatch = 336): 0.7997679656043131
SVGP accuracy (minibatch = 391): 0.7997679656043131
SVGP accuracy (minibatch = 445): 0.7997679656043131
SVGP accuracy (minibatch = 500): 0.7999044564253054
Logistic Regression accuracy: 0.8004504197092746
MLP accuracy: 0.7997679656043131


In [ ]:
plt.plot(mb_sizes, test_accuracies)
plt.show()
plt.plot(mb_sizes, -fairness_scores)
plt.show()

NameError: ignored